In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
rules = {}
parts = []
parsing_rules = True
for l in lines:
    if l == "":
        parsing_rules = False
        continue
    if parsing_rules:
        l = l.split("{")
        rule_name = l[0]
        rule = [r for r in l[1][:-1].split(",")]
        parsed_rules = []
        for r in rule:
            r = r.split(":")
            if len(r) > 1:
                comp = (r[0][0], r[0][1], int(r[0][2:]))
                parsed_rules.append((comp, r[1]))
            else:
                parsed_rules.append(r[0])
        # print(f"{rule_name}: {parsed_rules}")
        rules[rule_name] = parsed_rules
    else:
        l = l[1:-1].split(",")
        l_d = {}
        for e in l:
            e = e.split("=")
            l_d[e[0]] = int(e[1])
        parts.append(l_d)
# rules

In [ ]:
def apply_rule(part, rule):
    for i in range(0, len(rule) - 1):
        comp = rule[i][0]
        var = comp[0]
        if comp[1] == "<":
            if part[var] < comp[2]:
                return rule[i][1]
            else:
                continue
        else:
            if part[var] > comp[2]:
                return rule[i][1]
            else:
                continue
    return rule[-1]


def handle_part(part, rules):
    current_rule = "in"
    while True:
        rule_res = apply_rule(part, rules[current_rule])
        if rule_res in ["A", "R"]:
            return rule_res
        else:
            current_rule = rule_res


rating_sum = 0
for p in parts:
    if handle_part(p, rules) == "A":
        rating_sum += sum([v for v in p.values()])
rating_sum

part 2

In [ ]:
p2rules = rules.copy()


def invert_cond(condition):
    if condition[1] == "<":
        return (condition[0], ">", condition[2] - 1)
    else:
        return (condition[0], "<", condition[2] + 1)


# Think of the rules as a tree
# Each rule is a decision based on the condition to go either left or right until a leaf with A or R is reached
# When A is reached, all conditions to get there reduce the number of possibilities
# R is rejected

# search_accept_leafs starts at the first tree node
# Tree nodes are either a list, a tuple or a string
# If the string is A or R, it is a leaf, otherwise it is a reference to another tree node that can be fetched from the rules dict
# conditions collects all conditions until either A or R is reached in order
# in case of A, it returns the conditions needed to get there


def search_accept_leafs(tree_node, conditions):
    # either leaf or reference to other tree node which continues the search with same conditions because nothing happened
    if isinstance(tree_node, str):
        if tree_node == "A":
            return [conditions]
        elif tree_node == "R":
            return []
        else:
            return search_accept_leafs(p2rules[tree_node], conditions)
    else:
        # A middle node with a left and a right condition (which is the left condition inverted)
        left_cond = tree_node[0][0]
        right_cond = invert_cond(tree_node[0][0])
        remaining_right = tree_node[1:]
        # There is the possibility of a string as right part or the remainder of a list
        # both can be passed to search_accept_leafs and are handled correctly
        # Explore both paths, attaching the respective condition to the recursive call
        if len(remaining_right) > 1:
            return search_accept_leafs(
                tree_node[0][1], conditions + [left_cond]
            ) + search_accept_leafs(tree_node[1:], conditions + [right_cond])
        else:
            return search_accept_leafs(
                tree_node[0][1], conditions + [left_cond]
            ) + search_accept_leafs(tree_node[1], conditions + [right_cond])


# p2rules["test1"] = [(('s', '<', 5), 'A'), 'R']
# p2rules["test2"] = [(('a', '<', 6), 'test1'), 'R']
# p2rules["test3"] = [(('x', '<', 6), 'test2'), (('m', '<', 6), 'test1'), 'A']
# search_accept_leafs(p2rules["test3"], [])

# print(p2rules["in"])

In [ ]:
# go through all conditions and reduce the possible range of values according to condition
# then multiply all of the remaining possibilities
def get_remaining_pos(ruleset):
    x_from = 1
    x_to = 4000
    m_from = 1
    m_to = 4000
    a_from = 1
    a_to = 4000
    s_from = 1
    s_to = 4000
    for rule in ruleset:
        if rule[1] == "<":
            if rule[0] == "x":
                x_to = min(x_to, rule[2] - 1)
            if rule[0] == "m":
                m_to = min(m_to, rule[2] - 1)
            if rule[0] == "a":
                a_to = min(a_to, rule[2] - 1)
            if rule[0] == "s":
                s_to = min(s_to, rule[2] - 1)
        else:
            if rule[0] == "x":
                x_from = max(x_from, rule[2] + 1)
            if rule[0] == "m":
                m_from = max(m_from, rule[2] + 1)
            if rule[0] == "a":
                a_from = max(a_from, rule[2] + 1)
            if rule[0] == "s":
                s_from = max(s_from, rule[2] + 1)

    # print(x_from, x_to, m_from, m_to, a_from, a_to, s_from,s_to)
    remaining_pos = (
        (x_to - x_from + 1)
        * (m_to - m_from + 1)
        * (a_to - a_from + 1)
        * (s_to - s_from + 1)
    )
    return remaining_pos


# get_remaining_pos([("s", "<", 1351), ("a", "<", 2006), ("x", "<", 1416)])
sum([get_remaining_pos(r) for r in search_accept_leafs(p2rules["in"], [])])